In [1]:
import sys
sys.path.append('../')

import json
from EM import NiftiManager, Evaluate, EM, ElastixTransformix, FileManager
from glob import glob
import os
from tqdm import tqdm

# To allow auto reload to this notebook after modifying any external file imported
%load_ext autoreload
%autoreload 2

In [2]:
NM    = NiftiManager()
EVAL  = Evaluate()
ET    = ElastixTransformix()
FM    = FileManager()

In [3]:
print(ET.excute_cmd('elastix --version'))

elastix version: 4.700



Loading the Registration parameters

In [4]:
os.listdir('./registration-parameters')

['Par0010affine.txt', 'Par0010bspline.txt']

In [5]:
reg_params = '-p "./registration-parameters/Par0010affine.txt" -p "./registration-parameters/Par0010bspline.txt"'

Loading the dataset

In [6]:
train_path = '../TrainingValidationTestSets/Training_Set'
valid_path = '../TrainingValidationTestSets/Validation_Set'
test_path  = '../TrainingValidationTestSets/Test_Set'

In [7]:
train_volumes = sorted(glob(os.path.join(train_path, "***", "IBSR_*.nii.gz"), recursive=True))
train_volumes = [path for path in train_volumes if 'seg' not in path]

train_labels  = sorted(glob(os.path.join(train_path, "***", "IBSR_*_seg.nii.gz"), recursive=True))

In [8]:
valid_volumes = sorted(glob(os.path.join(valid_path, "***", "IBSR_*.nii.gz"), recursive=True))
valid_volumes = [path for path in valid_volumes if 'seg' not in path]

valid_labels  = sorted(glob(os.path.join(valid_path, "***", "IBSR_*_seg.nii.gz"), recursive=True))

In [9]:
valid_volumes

['../TrainingValidationTestSets/Validation_Set\\IBSR_11\\IBSR_11.nii.gz',
 '../TrainingValidationTestSets/Validation_Set\\IBSR_12\\IBSR_12.nii.gz',
 '../TrainingValidationTestSets/Validation_Set\\IBSR_13\\IBSR_13.nii.gz',
 '../TrainingValidationTestSets/Validation_Set\\IBSR_14\\IBSR_14.nii.gz',
 '../TrainingValidationTestSets/Validation_Set\\IBSR_17\\IBSR_17.nii.gz']

In [10]:
test_volumes = sorted(glob(os.path.join(test_path, "***", "IBSR_*.nii.gz"), recursive=True))
test_volumes = [path for path in test_volumes if 'seg' not in path]

Registration and label propagation (all training to all targets)

In [31]:
# we will assume the valid set is the test to be able to evaluate our approach for now

for fixed_volume, fixed_label in zip(valid_volumes, valid_labels): 
    image_id = fixed_volume.replace('\\', '/').split("/")[-1].split(".")[0]

    print(f"------------- Fixed {image_id} -------------")
    
    for train_volume, train_label in tqdm(zip(train_volumes, train_labels)):
        reg_moving_name = train_volume.replace("\\", "/").split("/")[-1].split(".")[0]

        # register an example train volume to the test space
        ET.register_elastix(
            fixed_path = fixed_volume, 
            moving_path = train_volume,
            reg_params = reg_params,
            create_dir_callback = FM.create_directory_if_not_exists,
            excute_cmd_callback = ET.excute_cmd)
    
        # perform label propagation to test space
        ET.label_propagation_transformix(
            fixed_path = fixed_volume, 
            moving_path = train_label, 
            input_label = train_label,
            transform_path = f'output/images/output_{image_id}/{reg_moving_name}/TransformParameters.1.txt',
            replace_text_in_file_callback = FM.replace_text_in_file,
            create_dir_callback = FM.create_directory_if_not_exists,
            excute_cmd_callback = ET.excute_cmd)
    

------------- Fixed IBSR_11 -------------


10it [24:29, 146.91s/it]


------------- Fixed IBSR_12 -------------


10it [24:24, 146.42s/it]


------------- Fixed IBSR_13 -------------


10it [41:36, 249.68s/it]


------------- Fixed IBSR_14 -------------


10it [24:21, 146.19s/it]


------------- Fixed IBSR_17 -------------


10it [24:59, 149.90s/it]


Note: output folder has been renamed to output_base for the previous experiment.

As we observed different voxel sizes accross the dataset, we defined a dataset structure for handling the registration paths, where we will register similar modalities with similar voxel sizes togather to avoid sampling. In `TrainingValidationTestSets\description.json`, we clustered each given volume to a group and each group will be registered togather.

In [12]:
with open('../TrainingValidationTestSets/description.json', 'r') as json_file:
        dataset = json.loads(json_file.read())
# FM.pprint_objects(dataset)

for modality in dataset:
    print(f"--- Processing data cluster with voxel sizes {modality}. ---")
    # we get the data of the same modality
    data_cluster = dataset[modality]

    # defining base paths
    train_base = '../TrainingValidationTestSets/Training_Set'
    valid_base = '../TrainingValidationTestSets/Validation_Set'
    test_base  = '../TrainingValidationTestSets/Test_Set'
    
    # register the data of the same modality to the target
    train_group  = data_cluster['train']  # to be modified for inference
    target_group = data_cluster['valid']  # to be modified for inference
    target_base  = valid_base             # to be modified for inference

    for fixed in target_group:
        print(f">> Processing fixed {fixed}...")
        fixed_volume = os.path.join(target_base, fixed, fixed + ".nii.gz")
        fixed_label  = os.path.join(target_base, fixed, fixed + "_seg.nii.gz")
        # print(fixed_volume, fixed_label)
        
        for train in tqdm(train_group):
            # to be modified for inference: if we merge train and val, train_base has to be altered with a condition to point to both directories
            train_volume = os.path.join(train_base, train, train + ".nii.gz") 
            train_label  = os.path.join(train_base, train, train + "_seg.nii.gz") 

            # register an example train volume to the test space
            ET.register_elastix(
                fixed_path = fixed_volume, 
                moving_path = train_volume,
                reg_params = reg_params,
                create_dir_callback = FM.create_directory_if_not_exists,
                excute_cmd_callback = ET.excute_cmd)

            # perform label propagation to test space
            ET.label_propagation_transformix(
                fixed_path = fixed_volume, 
                moving_path = train_label, 
                input_label = train_label,
                transform_path = f'output/images/output_{fixed}/{train}/TransformParameters.1.txt',
                replace_text_in_file_callback = FM.replace_text_in_file,
                create_dir_callback = FM.create_directory_if_not_exists,
                excute_cmd_callback = ET.excute_cmd)
    print("")

--- Processing data cluster with voxel sizes (0.9375, 1.5, 0.9375, 0.0). ---
>> Processing fixed IBSR_13...


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [03:27<00:00, 41.45s/it]


>> Processing fixed IBSR_14...


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [03:28<00:00, 41.62s/it]



--- Processing data cluster with voxel sizes (1.0, 1.5, 1.0, 0.0). ---
>> Processing fixed IBSR_11...


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [02:04<00:00, 41.45s/it]


>> Processing fixed IBSR_12...


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [02:02<00:00, 40.79s/it]



--- Processing data cluster with voxel sizes (0.8370536, 1.5, 0.8370536, 0.0). ---
>> Processing fixed IBSR_17...


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:21<00:00, 40.92s/it]